<a href="https://colab.research.google.com/github/Sean-1005/JSC270_A4/blob/main/JSC270_A4Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 1** Sentiment Analysis with a Common Twitter Dataset

In [1]:
from google.colab import files
import pandas as pd
import numpy as np
import io

In [ ]:
# upload the training dataset (covid-tweets-train.csv)
df_train = files.upload()

Saving covid-tweets-train.csv to covid-tweets-train (3).csv


In [20]:
# upload the testing dataset (covid-tweets-test.csv)
df_test = files.upload()

Saving covid-tweets-test.csv to covid-tweets-test (2).csv


In [21]:
# Read the csv files
df_train = pd.read_csv(io.BytesIO(df_train['covid-tweets-train.csv']))
df_test = pd.read_csv(io.BytesIO(df_test['covid-tweets-test.csv']))

In [22]:
# Check invalid values
df_train.info()
df_train['Sentiment'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41155 entries, 0 to 41154
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     41155 non-null  int64 
 1   OriginalTweet  41155 non-null  object
 2   Sentiment      41153 non-null  object
dtypes: int64(1), object(2)
memory usage: 964.7+ KB


2            18042
0            15397
1             7712
 PA"             1
 England"        1
Name: Sentiment, dtype: int64

There is 2 Nan and 2 invalid values in the df_train['Sentiment'].

In [23]:
#cleaning data
# Find the row number of the four invalid values for Nan and Sentiment PA", and England". 
index = []
for i in range(len(df_train['Sentiment'])):
  if df_train['Sentiment'][i] != '0' and df_train['Sentiment'][i] != '1' and df_train['Sentiment'][i] != '2':
    index.append(i)

print(index)

[15269, 18450, 21383, 36128]


In [17]:
# print the sentiment with corresponding row number
print(df_train['Sentiment'][15269])
print(df_train['Sentiment'][18450])
print(df_train['Sentiment'][21383])
print(df_train['Sentiment'][36128])

# print the tweet with corresponding row number
print("The 21383 tweet:", df_train['OriginalTweet'][18450])
print("The 36128 tweet", df_train['OriginalTweet'][36128])

nan
 PA"
nan
 England"

22250,67202,Cranberry Township



39929,84881,Worthing


In [24]:
# Label the tweets based on theor content
df_train.loc[18450, 'Sentiment'] = '0'
df_train.loc[36128, 'Sentiment'] = '2'
# drop the Nan rows
df_train.dropna(subset = ['Sentiment'], inplace = True)
df_train['Sentiment']  = df_train['Sentiment'].astype(int)

(A) Consider the training data. What is the balance between the three classes? In other words, what
proportion of the observations (in the training set) belong to each class?


In [ ]:
df_train.Sentiment.value_counts()
df_train.info()
df_train["Sentiment"].value_counts()

The proportion of 'positive' is $\frac{18043}{41153}=43.84\%$. 

The proportion of 'neutral' is $\frac{7712}{41153}=18.74\%$.

The proportion of 'negative' is $\frac{15398}{41153}=37.42\%$.

(B) Tokenize the tweets.

In [ ]:
# Create a new column in df_train and df_test that contains token lists
def tokenize(df):
  token = []
  for tweet in df['OriginalTweet']:
    token.append(tweet.split())
  return token

df_train['tokens'] = tokenize(df_train)
df_test['tokens'] = tokenize(df_test)

(C) Remove URL tokens

In [ ]:
import re

def no_links(df):
  tokens_no_links = []
  for tweet in df['tokens']:
    tokens_no_links.append([re.sub('^http.*$','', t) for t in tweet])
  return tokens_no_links

df_train['tokens'] = no_links(df_train)
df_test['tokens'] = no_links(df_test)

(D) Remove all punctuation and convert all the tokens to lowercase


In [ ]:
# Remove all punctuations
temp_train = []
temp_test = []
tokens_no_punct_train = []
tokens_no_punct_test = []

for row in df_train["tokens"]:
  temp_train.append([re.sub('[^\w\s]','', t) for t in row])

for row in temp_train:
  tokens_no_punct_train.append([w for w in row if w != ''])

for row in df_test["tokens"]:
  temp_test.append([re.sub('[^\w\s]','', t) for t in row])

for row in temp_test:
  tokens_no_punct_test.append([w for w in row if w != ''])

# Covert all row into lowercase
tokens_lowercase_train = []
tokens_lowercase_test = []
for row in tokens_no_punct_train:
  tokens_lowercase_train.append([t.lower() for t in row])

for row in tokens_no_punct_test:
  tokens_lowercase_test.append([t.lower() for t in row])

df_train["tokens"] = tokens_lowercase_train
df_test["tokens"] = tokens_lowercase_test


（E）Stemming using PorterStemmer


In [ ]:
from nltk.stem.porter import *

stemmer = PorterStemmer()

def stem(df):
  stemed = []
  for tweet in df['tokens']:
    stemed.append([stemmer.stem(w) for w in tweet])
  return stemed

df_train["tokens_stem"] = stem(df_train)
df_test["tokens_stem"] = stem(df_test)

(F) Remove teh first 100 stopwords in the lists of stopwords fromnltk

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

sw = stopwords.words('english')[:100]

tokens_no_sw_train = []
tokens_no_sw_test = []

for row in df_train['tokens_stem']:
  tokens_no_sw_train.append([w for w in row if w not in sw])

df_train['tokens_stem'] = tokens_no_sw_train

for row in df_test['tokens_stem']:
  tokens_no_sw_test.append([w for w in row if w not in sw])

df_test['tokens_stem'] = tokens_no_sw_test

(G) Convert lists of words into vectors of word counts

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split


# Separate labels from features, converting to numpy arrays
X_train, y_train = df_train['tokens_stem'].to_numpy(), df_train['Sentiment'].to_numpy()
X_test, y_test = df_test['tokens_stem'].to_numpy(), df_test['Sentiment'].to_numpy()

def override_fcn(doc):
  # We expect a list of tokens as input
  return doc

vocab = (np.unique(np.concatenate(X_train))).shape[0]

# Count Vectorizer
count_vec = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

# Remember this output is a Scipy Sparse Array
X = np.concatenate((X_train, X_test), axis=0)
counts = count_vec.fit_transform(X)

In [ ]:
print(len(count_vec.vocabulary_)) # length of the vocabulary

(H) Fit Naive Bayes model using count vectors

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
X_train, X_test = counts[:np.size(X_train), : ], counts[np.size(X_train):, :]

nb = MultinomialNB()
nb.fit(X_train, y_train)

predict = nb.predict(X_test)

print('Test accuracy with simple Naive Bayes:',accuracy_score(y_test, predict))

In [ ]:
# FInd the top 5 words for each class
positive = []
neutral = []
negative = []

X, y = df_train['tokens_stem'].to_numpy(), df_train['Sentiment'].to_numpy()

for i in range(len(X)):
  if y[i] == 0:
    negative.append(X[i])
  elif y[i] == 1:
    neutral.append(X[i])
  else:
    positive.append(X[i])

count_vec_postop = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

# Remember this output is a Scipy Sparse Array
count_vec_postop.fit_transform(positive)

# Print the names of each of the features (1000 total))
token_freq_p = count_vec_postop.vocabulary_

count_vec_neutop = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

# Remember this output is a Scipy Sparse Array
count_vec_neutop.fit_transform(neutral)

# Print the names of each of the features (1000 total))
token_freq_neu = count_vec_neutop.vocabulary_

count_vec_negtop = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

# Remember this output is a Scipy Sparse Array
count_vec_negtop.fit_transform(negative)

# Print the names of each of the features (1000 total))
token_freq_n = count_vec_negtop.vocabulary_


sort_orders_p = sorted(token_freq_p.items(), key=lambda x: x[1], reverse=True)
sort_orders_neu = sorted(token_freq_neu.items(), key=lambda x: x[1], reverse=True)
sort_orders_n = sorted(token_freq_n.items(), key=lambda x: x[1], reverse=True)

print("Top 5 words for 2: ", sort_orders_p[:5])
print("Top 5 words for 1: ", sort_orders_neu[:5])
print("Top 5 words for 0: ", sort_orders_n[:5])

(I) ROC curve

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(y_test, predict, pos_label = 1)

plt.plot(fpr,tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operator Characteristic (ROC) Curve for our model')
plt.show()

J) Fit the Naive Bayes model using TF-IDF vectors

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split


# Separate labels from features, converting to numpy arrays
X_train, y_train = df_train['tokens_stem'].to_numpy(), df_train['Sentiment'].to_numpy()
X_test, y_test = df_test['tokens_stem'].to_numpy(), df_test['Sentiment'].to_numpy()

def override_fcn(doc):
  # We expect a list of tokens as input
  return doc

vocab = (np.unique(np.concatenate(X_train))).shape[0]

# Count Vectorizer
count_vec = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

# Remember this output is a Scipy Sparse Array
X = np.concatenate((X_train, X_test), axis=0)
counts = count_vec.fit_transform(X)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
######for training data
tfidf = TfidfTransformer()
tfs1 = tfidf.fit_transform(counts)

# Let's use the TFIDF counts for modelling
X = tfs1.toarray()
X_train, X_test = X[:np.size(X_train), : ], X[np.size(X_train):, :]

# Let's fit the Naive Bayes model to our training data
nb = MultinomialNB()
# Fit model to training data
nb.fit(X_train, y_train)
# Predict on test data
y_preds = nb.predict(X_test)
X_test, y_test = df_test['tokens_stem'].to_numpy(), df_test['Sentiment'].to_numpy()
print('Test accuracy with simple Naive Bayes:',accuracy_score(y_test,y_preds))

(K) Fit teh Naive Bayes model using lemmatization instead of Stemming

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

lem_tokens = []
for row in df_train['tokens']:
  lem_tokens.append([lemmatizer.lemmatize(t) for t in row])

df_train['tokens_lem'] = lem_tokens

lem_tokens2 = []
for row in df_test['tokens']:
  lem_tokens2.append([lemmatizer.lemmatize(t) for t in row])

df_test['tokens_lem'] = lem_tokens2


In [ ]:
sw_lem = stopwords.words('english')[:100]

tokens_no_sw_train_lem = []
tokens_no_sw_test_lem = []

for row in df_train['tokens_lem']:
  tokens_no_sw_train_lem.append([w for w in row if w not in sw_lem])

df_train['tokens_lem'] = tokens_no_sw_train_lem

for row in df_test['tokens_lem']:
  tokens_no_sw_test_lem.append([w for w in row if w not in sw_lem])

df_test['tokens_lem'] = tokens_no_sw_test_lem

In [ ]:
X_train, y_train = df_train['tokens_lem'].to_numpy(), df_train['Sentiment'].to_numpy()
X_test, y_test = df_test['tokens_lem'].to_numpy(), df_test['Sentiment'].to_numpy()

def override_fcn(doc):
  # We expect a list of tokens as input
  return doc

# Count Vectorizer
count_vec = CountVectorizer(
    analyzer='word',
    tokenizer= override_fcn,
    preprocessor= override_fcn,
    token_pattern= None,
    max_features = 1000)

# Remember this output is a Scipy Sparse Array
X = np.concatenate((X_train, X_test), axis=0)
counts = count_vec.fit_transform(X)

In [ ]:
tfidf = TfidfTransformer()
tfs1 = tfidf.fit_transform(counts)

# Let's use the TFIDF counts for modelling
X = tfs1.toarray()
X_train, X_test = X[:np.size(X_train), : ], X[np.size(X_train):, :]

# Let's fit the Naive Bayes model to our training data
nb = MultinomialNB()
# Fit model to training data

nb.fit(X_train, y_train)
# Predict on test data
y_preds = nb.predict(X_test)
X_test, y_test = df_test['tokens_lem'].to_numpy(), df_test['Sentiment'].to_numpy()
print('Test accuracy with simple Naive Bayes:',accuracy_score(y_test,y_preds))